## Question 1

In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/seacevedo/module_3/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

In [3]:
v

array([ 7.82226250e-02, -4.01311591e-02,  3.86136211e-02, -1.78957722e-04,
        8.92346650e-02, -5.04591241e-02, -1.05026914e-02,  3.71055864e-02,
       -4.18714248e-02,  3.48084830e-02, -1.20701939e-02, -2.36942377e-02,
        3.87899950e-02,  1.60988048e-02,  3.50747369e-02,  3.04751098e-03,
        5.79672456e-02, -4.10627276e-02, -3.41552831e-02, -2.56396048e-02,
       -3.55263948e-02,  1.42907910e-02, -1.62800215e-02,  3.21446210e-02,
       -4.66897450e-02,  7.89185762e-02,  4.90160882e-02,  1.56761073e-02,
       -1.69110261e-02,  2.26482358e-02,  5.60206175e-02, -3.98361273e-02,
        6.77409843e-02, -1.20210117e-02,  1.12624269e-03, -1.94394644e-02,
       -2.65951380e-02,  1.06177889e-02,  1.69687588e-02,  1.13487830e-02,
       -2.97063291e-02,  5.25258631e-02, -1.41453361e-02,  4.61699814e-02,
        1.17066242e-02, -2.38053091e-02, -6.32557869e-02, -1.92042124e-02,
       -7.10594654e-03,  3.24167795e-02,  2.49617975e-02, -5.27500082e-03,
        2.01149415e-02, -

In [4]:
7.82226250e-02

0.078222625

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
new_docs = []
for doc in documents:
    if doc['course'] == "machine-learning-zoomcamp":
        new_docs.append(doc)

In [7]:
len(new_docs)

375

## Question 2

In [ ]:
import numpy as np

embeddings = []

for doc in new_docs:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    v_qa = embedding_model.encode(qa_text)
    embeddings.append(v_qa)

In [ ]:
embeddings

In [ ]:
X = np.array(embeddings)

In [ ]:
X.shape

## Question 3

In [ ]:
np.max(X.dot(v))

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

## Question 4

In [ ]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')